# Alura Challenge BI 2ª Edição - Semana 1

## Resumo da Semana

Nesta primeira semana do desafio, a Alura Films deseja encontrar a seleção ideal de elenco e produção. Por isso, fui contratado para analisar uma base de dados com informações de 1000 filmes, de diversos gêneros, orçamentos e estrelas.

Esses filmes foram tirados da lista dos 1000 filmes com melhores notas dos usuários do site IMDb (Internet Movie Database).

A empresa forneceu dois arquivos do tipo .csv: um com informações sobre os filmes e outro com links para os Poster de cada filme. Para verificar os dados dos filmes, irei utilizar Python Pandas para fazer uma análise exploratória dos dados e, se necessário, tratar os dados que foram disponibilizados. Depois, irei desenvolver um dashboard para que a empresa possa, de forma rápida e eficiente, tomar decisões com base nesses dados.

Além dos dados, a empresa também enviou um documento de texto explicando as informações contidas nas colunas das bases de dados. Mesmo assim irei conduzir uma análise exploratória rápida para entender melhor os dados e também verificar se algum dado possa estar faltando.

## Análise dos dados dos filmes

### Importando

#### Biblioteca

In [77]:
import pandas as pd

!python --version
print(f'A versão de Pandas utilizada é {pd.__version__}')

Python 3.7.12
A versão de Pandas utilizada é 1.3.5


#### Dados


Os dados foram disponibilizados por meio do Google Drive. Primeiro inclui eles em um repositório do GitHub e depois importei eles para esta análise. Inicialmente irei tratar apenas o arquivo que contém as informações dos 1000 filmes, sem os posters.

In [78]:
url = 'https://raw.githubusercontent.com/vinicius-pf/BI_Challenge_2/main/Semana%201/dataset/Filmes.csv'
dados = pd.read_csv(url)

In [79]:
print(f'A base contém {dados.shape[0]} filmes e {dados.shape[1]} colunas')

A base contém 1000 filmes e 16 colunas


In [80]:
dados.head()

,Id_Title,Series_Title,Gross,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,Noofvotes
0,1,The Shawshank Redemption,"28,341,469",1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110
1,2,The Godfather,"134,966,411",1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367
2,3,The Dark Knight,"534,858,444",2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232
3,4,The Godfather: Part II,"57,300,000",1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952
4,5,12 Angry Men,"4,360,000",1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845


Como primeira etapa, imprimi as primeiras 5 linhas dos dados. Com isso percebi que:

1 -  Os dados estão em inglês. Para facilitar, os nomes das colunas serão traduzidos para o português. Além disso, os dados de 'Series_Title', 'Genre' e 'Overview', estão também em inglês. Fazer as traduções desses dados pode ser interessante para a Alura Films.

2 - Alguns filmes tem 2 ou mais 'Genre'. A empresa requisitou que fossem feitas análises quanto a rentabilidade de gêneros de filmes, então essa coluna poderá ser dividida para facilitar essa análise.

3 - A coluna 'Certificate' não tem uma explicação ainda. Acredito que seja a classificação indicativa do filme, porém, pesquisando no próprio IMDb <sup>[1](https://help.imdb.com/article/contribution/titles/certificates/GU757M8ZJ9ZPXB39?ref_=helpart_nav_27#)</sup>, não existe nenhum país que utilize as letras que apareceram. Devo continuar explorando essa coluna.

4 - A coluna 'IMDB_Rating' corresponde aos votos dos usuários do site IMDb, enquanto a coluna 'Meta_Score' corresponde à pontuação no site da Metacritic.<sup>[2](https://www.quora.com/What-does-the-Metascore-score-on-IMDb-mean)</sup>

5 - Os valores de 'Gross' são estimados em dólares <sup>[3](https://help.imdb.com/article/imdb/discover-watch/box-office-faq/G4UCJ3GMFX6F23ZX#)</sup>. Para informação mais regionalizada, irei criar uma coluna utilizando a cotação do dólar no dia 14/02/2022 para saber o valor em reais.

In [81]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id_Title       1000 non-null   int64  
 1   Series_Title   1000 non-null   object 
 2   Gross          989 non-null    object 
 3   Released_Year  1000 non-null   int64  
 4   Certificate    1000 non-null   object 
 5   Runtime        1000 non-null   object 
 6   Genre          1000 non-null   object 
 7   IMDB_Rating    1000 non-null   float64
 8   Overview       1000 non-null   object 
 9   Meta_score     843 non-null    float64
 10  Director       1000 non-null   object 
 11  Star1          1000 non-null   object 
 12  Star2          1000 non-null   object 
 13  Star3          1000 non-null   object 
 14  Star4          1000 non-null   object 
 15  Noofvotes      1000 non-null   int64  
dtypes: float64(2), int64(3), object(11)
memory usage: 125.1+ KB


Após a primeira verificação das linhas, imprimi as informações das colunas e descobri que:

1 - As colunas 'Gross' e 'Meta_Score' não estão com informações para todos os filmes. Precisarei analisar com mais calma as colunas para saber o que deverá ser feito.

2 - A coluna 'Release_Year' está no formato 'int64'. Como o desejo da empresa é receber um dashboard, para facilitar a criação de uma possível linha do tempo, essa coluna deve estar no formato 'datetime'. Essa troca poderá ser feita na plataforma de criação de dashboards, então não mudarei por enquanto.

### Aprofundando

Após as análises iniciais, encontrei 6 pontos que necessitam de minha atenção. me aprofundarei em cada ponto para que consiga definir um meio de ação, seja dentro das funcionalidades do python pandas ou por meio de outra ferramenta.

#### 1 - Traduzindo as colunas

Como primeira etapa, traduzirei os títulos de colunas para o português. Desejando manter o dataframe original, irei criar um outro dataframe para fazer as alterações. Caso haja algum problema, poderei voltar ao original sem problemas.

In [82]:
dados_traduzidos = dados.copy()

Para traduzir com mais clareza, criarei um dicionário com as traduções necessárias e depois irei trocar os nomes das colunas. Para manter a relação entre os dois arquivos, não irei traduzir a coluna 'ID_Title'. A coluna 'Certificate' será traduzida para 'Classificação Indicativa'.

In [83]:
dados_traduzidos.columns

Index(['Id_Title', 'Series_Title', 'Gross', 'Released_Year', 'Certificate',
       'Runtime', 'Genre', 'IMDB_Rating', 'Overview', 'Meta_score', 'Director',
       'Star1', 'Star2', 'Star3', 'Star4', 'Noofvotes'],
      dtype='object')

In [84]:
traducoes = {'Series_Title' : 'Título', 'Gross' : 'Receita', 'Released_Year' : 'Ano de lançamento', 'Certificate' : 'Classificação indicativa',
       'Runtime' : 'Duração do filme', 'Genre' : 'Gênero', 'IMDB_Rating' : 'Nota IMDd', 'Overview' : 'Resumo', 'Meta_score' : 'Nota Metacrict', 'Director' : 'Diretor',
       'Star1' : 'Estrela 1', 'Star2' : 'Estrela 2', 'Star3' : 'Estrela 3', 'Star4' : 'Estrela 4', 'Noofvotes' : 'Número de Votos'}

In [85]:
dados_traduzidos.rename(traducoes, axis=1, inplace= True)

In [86]:
dados_traduzidos.head()

,Id_Title,Título,Receita,Ano de lançamento,Classificação indicativa,Duração do filme,Gênero,Nota IMDd,Resumo,Nota Metacrict,Diretor,Estrela 1,Estrela 2,Estrela 3,Estrela 4,Número de Votos
0,1,The Shawshank Redemption,"28,341,469",1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110
1,2,The Godfather,"134,966,411",1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367
2,3,The Dark Knight,"534,858,444",2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232
3,4,The Godfather: Part II,"57,300,000",1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952
4,5,12 Angry Men,"4,360,000",1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845


#### 2 - Tratando a coluna de gêneros

Uma das métricas pedidas pela empresa foi a de 'Gêneros mais rentáveis'. Para descobrir isso, precisarei utilizar a coluna 'Gêneros'. No entanto, alguns filmes possuem mais de um gênero, o que dificultaria a análise por meio de softwares. Para facilitar esse trabalho, criarei outras colunas que terão os gêneros dos filmes, cada uma com apenas 1 gênero.

In [87]:
dados_traduzidos['Gênero'].unique()[:5]

array(['Drama', 'Crime, Drama', 'Action, Crime, Drama',
       'Action, Adventure, Drama', 'Biography, Drama, History'],
      dtype=object)

Todos os 5 primeiros filmes são considerados do gênero 'Drama', porém, sem fazer o tratamento correto dos dados, os valores de receita não serão corretamente calculados. Para isso, criarei colunas novas para cada gênero, esperando assim que esse problema seja resolvido.

Primeiro, irei criar um DataFrame temporário para receber as colunas que irão separar os gêneros, já botando os gêneros nas colunas

In [88]:
temp = dados_traduzidos['Gênero'].str.split(', ', expand = True)
temp = pd.DataFrame(temp)
temp.head()

,0,1,2
0,Drama,None,None
1,Crime,Drama,None
2,Action,Crime,Drama
3,Crime,Drama,None
4,Crime,Drama,None


Irei agora renomear as colunas, preparando o Dataframe temporário para a concatenação.

In [89]:
colunas_novas = {0 : 'Gênero 1', 1 : 'Gênero 2', 2 : 'Gênero 3'}
temp.rename(colunas_novas, axis=1, inplace = True)
temp.head()

,Gênero 1,Gênero 2,Gênero 3
0,Drama,None,None
1,Crime,Drama,None
2,Action,Crime,Drama
3,Crime,Drama,None
4,Crime,Drama,None


Por último, concatenarei os dois Dataframes, concluindo assim o trabalho.

In [90]:
dados_novos = pd.concat([dados_traduzidos,temp], axis = 1)
dados_novos.head(3)

,Id_Title,Título,Receita,Ano de lançamento,Classificação indicativa,Duração do filme,Gênero,Nota IMDd,Resumo,Nota Metacrict,Diretor,Estrela 1,Estrela 2,Estrela 3,Estrela 4,Número de Votos,Gênero 1,Gênero 2,Gênero 3
0,1,The Shawshank Redemption,"28,341,469",1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,Drama,None,None
1,2,The Godfather,"134,966,411",1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,Crime,Drama,None
2,3,The Dark Knight,"534,858,444",2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,Action,Crime,Drama


Após a união, percebi que é possível traduzir os gêneros dos filmes de maneira mais eficaz. Primeiro irei verificar quais os gêneros dos filmes na base de dados.

In [91]:
dados_novos['Gênero 1'].unique()

array(['Drama', 'Crime', 'Action', 'Biography', 'Western', 'Comedy',
       'Adventure', 'Animation', 'Horror', 'Mystery', 'Film-Noir',
       'Fantasy', 'Family', 'Thriller'], dtype=object)

In [92]:
dados_novos['Gênero 2'].unique()

array([None, 'Drama', 'Crime', 'Adventure', 'Romance', 'Sci-Fi', 'War',
       'Family', 'Music', 'Comedy', 'Mystery', 'Biography', 'Action',
       'Western', 'Thriller', 'Horror', 'Musical', 'Film-Noir', 'Fantasy',
       'Sport', 'History'], dtype=object)

In [93]:
dados_novos['Gênero 3'].unique()

array([None, 'Drama', 'History', 'Sci-Fi', 'Fantasy', 'Thriller',
       'Family', 'Romance', 'Mystery', 'Music', 'War', 'Adventure',
       'Horror', 'Sport', 'Comedy', 'Film-Noir', 'Crime', 'Western',
       'Musical', 'Biography'], dtype=object)

Em seguida, irei criar um dicionário com as traduções.

In [94]:
generos_pt = {
    'Drama' : 'Drama', 
    'Crime' : 'Crime', 
    'Action' : 'Ação', 
    'Biography' : 'Biografia', 
    'Western' : 'Faroeste', 
    'Comedy' : 'Comédia',
    'Adventure' : 'Aventura', 
    'Animation' : 'Animação',
    'Horror' : 'Terror',
    'Mystery' : 'Mistério',
    'Film-Noir' : 'Noir',
    'Fantasy' : 'Fantasia', 
    'Family' : 'Família', 
    'Thriller' : 'Suspense',
    'Romance' : 'Romance', 
    'Sci-Fi' : 'Ficção Científica',
    'War' : 'Guerra', 
    'Music' : 'Musical',
    'Musical' : 'Musical',
    'Sport' : 'Esportes', 
    'History' : 'História'
}

Nessa tradução, percebi que há dois gêneros que podem significar o mesmo gênero: Music e Musical. Por isso traduzi os dois para o mesmo termo: Musical. Após isso, poderei traduzir os gêneros nas colunas novas.

In [95]:
dados_novos = dados_novos.replace(generos_pt)
dados_novos.head()

,Id_Title,Título,Receita,Ano de lançamento,Classificação indicativa,Duração do filme,Gênero,Nota IMDd,Resumo,Nota Metacrict,Diretor,Estrela 1,Estrela 2,Estrela 3,Estrela 4,Número de Votos,Gênero 1,Gênero 2,Gênero 3
0,1,The Shawshank Redemption,"28,341,469",1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,Drama,None,None
1,2,The Godfather,"134,966,411",1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,Crime,Drama,None
2,3,The Dark Knight,"534,858,444",2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,Ação,Crime,Drama
3,4,The Godfather: Part II,"57,300,000",1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,Crime,Drama,None
4,5,12 Angry Men,"4,360,000",1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,Crime,Drama,None


#### 3 - Explorando a classificação indicativa

Segundo a empresa, a coluna original 'Certificate' informa a classificação indicativa do filme. Porém, em pesquisa rápida no site do IMDb, não encontrei nenhum país que tem as classificações indicativas iguais as disponíveis na base de dados.

In [96]:
dados_novos['Classificação indicativa'].unique()

array(['A', 'UA', 'U', 'PG-13', 'R', '16', 'PG', 'G', 'Passed', 'TV-14',
       '10', '12', '14', 'TV-MA', 'Unrated', 'GP', '18', 'Not Rated',
       'Approved', 'not rated', 'Livre', 'TV-PG', '12A', 'PG-12',
       'Not rated', 'U/A'], dtype=object)

Utilizando essa informação, percebi que as classificações indicativas estão misturadas entre países. Existem classificações indicativas do Brasil, dos EUA, do Reino Unido e de outros países.

Independente do país, parecem haver alguns valores duplicados em 'Not Rated', 'not rated' e 'Not rated. Todos esses deviam ser iguais, então usarei 'Not Rated'. Além deles, 'Unrated' parece ter o mesmo significado. Também parece haver um erro de digitação na classificação 'GP', e aprofundarei essa pesquisa.

##### Aprofundando na classificação 'GP'

Primeiramente verificarei os filmes com a classificação 'GP', para saber se foi um erro de digitação ou se há algum detalhe a mais.

In [97]:
classificação_gp = dados_novos.loc[dados_novos['Classificação indicativa'] == 'GP']
classificação_gp

,Id_Title,Título,Receita,Ano de lançamento,Classificação indicativa,Duração do filme,Gênero,Nota IMDd,Resumo,Nota Metacrict,Diretor,Estrela 1,Estrela 2,Estrela 3,Estrela 4,Número de Votos,Gênero 1,Gênero 2,Gênero 3
287,288,Le samouraï,"39,481",1967,GP,105 min,"Crime, Drama, Mystery",8.1,After professional hitman Jef Costello is seen...,NaN,Jean-Pierre Melville,Alain Delon,François Périer,Nathalie Delon,Cathy Rosier,45434,Crime,Drama,Mistério
991,992,Kelly's Heroes,"1,378,435",1970,GP,144 min,"Adventure, Comedy, War",7.6,A group of U.S. soldiers sneaks across enemy l...,50.0,Brian G. Hutton,Clint Eastwood,Telly Savalas,Don Rickles,Carroll O'Connor,45338,Aventura,Comédia,Guerra


Esses dois filmes possuem a classificação 'GP'. Essa classificação foi utilizada na década de 1970 pelo Motion Picture Association. Em 1972, porém, a classificação foi alterada de 'GP' para 'PG'<sup> [3](https://en.wikipedia.org/wiki/Motion_Picture_Association_film_rating_system#Replacement_of_the_Hays_Code)</sup>. No momento, não farei nenhuma alteração nessa informação. Se for necessário, irei unificar a coluna para classificações indicativas normalizadas no Brasil pelo Ministério da Justiça e Segurança Pública <sup>[4](https://www.gov.br/mj/pt-br/assuntos/seus-direitos/classificacao-1)</sup>.

##### Unificando as classificações 'Not Rated'

Para unificar as classificações 'Not Rated', preciso saber quantos filmes estão com as classificações similares.

In [98]:
dados_novos['Classificação indicativa'].value_counts()

U            237
A            198
UA           175
R            150
PG            51
PG-13         43
Passed        34
16            26
14            15
G             12
Approved      11
12            10
not rated      8
18             7
12A            4
TV-PG          3
TV-14          3
Not rated      3
GP             2
Not Rated      2
PG-12          1
TV-MA          1
Livre          1
Unrated        1
10             1
U/A            1
Name: Classificação indicativa, dtype: int64

Há 16 filmes com as classifições indicativas que devem ser alteradas. Primeiramente, transformarei as classificações dos filmes 'not rated'.

In [99]:
dados_novos.loc[dados_novos['Classificação indicativa'] == 'not rated']

,Id_Title,Título,Receita,Ano de lançamento,Classificação indicativa,Duração do filme,Gênero,Nota IMDd,Resumo,Nota Metacrict,Diretor,Estrela 1,Estrela 2,Estrela 3,Estrela 4,Número de Votos,Gênero 1,Gênero 2,Gênero 3
310,311,The Red Shoes,"10,900,000",1948,not rated,135 min,"Drama, Music, Romance",8.1,A young ballet dancer is torn between the man ...,NaN,Michael Powell,Emeric Pressburger,Anton Walbrook,Marius Goring,Moira Shearer,30935,Drama,Musical,Romance
379,380,Yeopgijeogin geunyeo,"365,429",2001,not rated,137 min,"Comedy, Drama, Romance",8.0,"A young man sees a drunk, cute woman standing ...",NaN,Jae-young Kwak,Tae-Hyun Cha,Jun Ji-Hyun,In-mun Kim,Song Wok-suk,45403,Comédia,Drama,Romance
451,452,Out of the Past,NaN,1947,not rated,97 min,"Crime, Drama, Film-Noir",8.0,A private eye escapes his past to run a gas st...,NaN,Jacques Tourneur,Robert Mitchum,Jane Greer,Kirk Douglas,Rhonda Fleming,32784,Crime,Drama,Noir
551,552,Hiroshima mon amour,"88,3",1959,not rated,90 min,"Drama, Romance",7.9,A French actress filming an anti-war film in H...,NaN,Alain Resnais,Emmanuelle Riva,Eiji Okada,Stella Dassas,Pierre Barbaud,28421,Drama,Romance,None
589,590,La grande bellezza,"2,852,400",2013,not rated,141 min,Drama,7.8,Jep Gambardella has seduced his way through th...,86.0,Paolo Sorrentino,Toni Servillo,Carlo Verdone,Sabrina Ferilli,Carlo Buccirosso,81125,Drama,None,None
601,602,Kokuhaku,"45,203,103",2010,not rated,106 min,"Drama, Thriller",7.8,A psychological thriller of a grieving mother ...,NaN,Tetsuya Nakashima,Takako Matsu,Yoshino Kimura,Masaki Okada,Yukito Nishii,35713,Drama,Suspense,None
619,620,Auf der anderen Seite,"741,283",2007,not rated,122 min,Drama,7.8,A Turkish man travels to Istanbul to find the ...,85.0,Fatih Akin,Baki Davrak,Nurgül Yesilçay,Tuncel Kurtiz,Nursel Köse,30827,Drama,None,None
636,637,Ondskan,"15,28",2003,not rated,113 min,Drama,7.8,A teenage boy expelled from school for fightin...,61.0,Mikael Håfström,Andreas Wilson,Henrik Lundström,Gustaf Skarsgård,Linda Zilliacus,35682,Drama,None,None


De acordo com o site, todos possuem a mesma classificação: 'Not Rated'. Assim esses podem ter as classificações alteradas. No entanto, percebi algumas discrepâncias com os valores de receita de alguns filmes, como 'Hiroshima mon amour' e 'Ondskan'. Esses valores são muito baixos para um filme e tem valores diferentes nos respectivos sites do IMDb. Outros filmes podem ter o mesmo problema então analisarei com mais calma na próxima seção.

In [100]:
dados_novos.loc[dados_novos['Classificação indicativa'] == 'Not rated']

,Id_Title,Título,Receita,Ano de lançamento,Classificação indicativa,Duração do filme,Gênero,Nota IMDd,Resumo,Nota Metacrict,Diretor,Estrela 1,Estrela 2,Estrela 3,Estrela 4,Número de Votos,Gênero 1,Gênero 2,Gênero 3
742,743,Detachment,"71,177",2011,Not rated,98 min,Drama,7.7,A substitute teacher who drifts from classroom...,52.0,Tony Kaye,Adrien Brody,Christina Hendricks,Marcia Gay Harden,Lucy Liu,77071,Drama,None,None
762,763,Death Note: Desu nôto,"29,667,169",2006,Not rated,126 min,"Crime, Drama, Fantasy",7.7,A battle between the world's two greatest mind...,NaN,Shûsuke Kaneko,Tatsuya Fujiwara,Ken'ichi Matsuyama,Asaka Seto,Yû Kashii,28630,Crime,Drama,Fantasia
866,867,Les yeux sans visage,"52,709",1960,Not rated,90 min,"Drama, Horror",7.7,A surgeon causes an accident which leaves his ...,90.0,Georges Franju,Pierre Brasseur,Alida Valli,Juliette Mayniel,Alexandre Rignault,27620,Drama,Terror,None


In [101]:
dados_novos.loc[dados_novos['Classificação indicativa'] == 'Unrated']

,Id_Title,Título,Receita,Ano de lançamento,Classificação indicativa,Duração do filme,Gênero,Nota IMDd,Resumo,Nota Metacrict,Diretor,Estrela 1,Estrela 2,Estrela 3,Estrela 4,Número de Votos,Gênero 1,Gênero 2,Gênero 3
240,241,Dil Chahta Hai,300,2001,Unrated,183 min,"Comedy, Drama, Romance",8.1,Three inseparable childhood friends are just o...,NaN,Farhan Akhtar,Aamir Khan,Saif Ali Khan,Akshaye Khanna,Preity Zinta,66803,Comédia,Drama,Romance


Explorando a base de dados, entendi que as diferenças na coluna podem ser unificadas para 'Not Rated'. Importante frisar que a classificação atual será a dos EUA, enquanto a classificação 'Unrated' é indiana. Isso não é um problema, pois futuramente essa coluna deverá ser unificada para a classificação brasileira.

In [102]:
unificacao_classificacao = {'Not rated': 'Not Rated', 'not rated': 'Not Rated', 'Unrated' : 'Not Rated'}

In [103]:
dados_novos = dados_novos.replace(unificacao_classificacao)
dados_novos['Classificação indicativa'].value_counts()

U            237
A            198
UA           175
R            150
PG            51
PG-13         43
Passed        34
16            26
14            15
Not Rated     14
G             12
Approved      11
12            10
18             7
12A            4
TV-14          3
TV-PG          3
GP             2
TV-MA          1
10             1
Livre          1
PG-12          1
U/A            1
Name: Classificação indicativa, dtype: int64

As classificações 'Passed' e 'Approved' parecem também ser redundantes, podendo ser simplificadas.

In [104]:
dados_novos.loc[dados_novos['Classificação indicativa'] == 'Approved'].head()

,Id_Title,Título,Receita,Ano de lançamento,Classificação indicativa,Duração do filme,Gênero,Nota IMDd,Resumo,Nota Metacrict,Diretor,Estrela 1,Estrela 2,Estrela 3,Estrela 4,Número de Votos,Gênero 1,Gênero 2,Gênero 3
307,308,Ace in the Hole,"3,969,893",1951,Approved,111 min,"Drama, Film-Noir",8.1,A frustrated former big-city journalist now st...,72.0,Billy Wilder,Kirk Douglas,Jan Sterling,Robert Arthur,Porter Hall,31568,Drama,Noir,None
309,310,The Third Man,"449,191",1949,Approved,104 min,"Film-Noir, Mystery, Thriller",8.1,Pulp novelist Holly Martins travels to shadowy...,97.0,Carol Reed,Orson Welles,Joseph Cotten,Alida Valli,Trevor Howard,158731,Noir,Mistério,Suspense
312,313,Rebecca,"4,360,000",1940,Approved,130 min,"Drama, Mystery, Romance",8.1,A self-conscious woman juggles adjusting to he...,86.0,Alfred Hitchcock,Laurence Olivier,Joan Fontaine,George Sanders,Judith Anderson,123942,Drama,Mistério,Romance
316,317,It Happened One Night,"4,360,000",1934,Approved,105 min,"Comedy, Romance",8.1,A renegade reporter and a crazy young heiress ...,87.0,Frank Capra,Clark Gable,Claudette Colbert,Walter Connolly,Roscoe Karns,94016,Comédia,Romance,None
440,441,Sweet Smell of Success,"7,704",1957,Approved,96 min,"Drama, Film-Noir",8.0,Powerful but unethical Broadway columnist J.J....,100.0,Alexander Mackendrick,Burt Lancaster,Tony Curtis,Susan Harrison,Martin Milner,28137,Drama,Noir,None


A classificação 'Approved' foi utilizada nos EUA a partir da criação do The Motion Picture Production Code, também conhecido como 'Hays Code', que surgiu em virtude do advento do som nos filmes<sup>[6](https://www.asu.edu/courses/fms200s/total-readings/MotionPictureProductionCode.pdf)</sup>. Entre outros pontos, o código definia que filmes não podiam mostrar crimes ou injustiças que levavam a se criar uma simpatia pelo criminoso. Os filmes que estavam de acordo com o novo código, eram então dados como 'Approved', enquanto os filmes que não o respeitavam não recebiam classificação indicativa. Além de 'Approved', também era dado o selo de 'Passed' pelo mesmo código. A diferença parece estar no ano de lançamento do filme, apesar de haver discrepâncias<sup>[7](https://movies.stackexchange.com/questions/65430/what-are-the-meanings-of-the-terms-passed-and-approved-with-regards-to-a-mov)</sup>. 

A partir da década de 1960, foi percebida a necessidade de atualização do código, criticado por ser rigoroso e até 'tendo o cheiro odioso de censura', e o processo de criação das classificações etárias começou a ser criado. Em 1968 o 'Motion Picture Association film rating system' foi desenvolvido para substituir o 'Hays Code'. Ao longo dos anos o novo sistema foi sendo atualizado e revisto, sendo utilizado até hoje em todas as produções cinematográficas americanas<sup>[3](https://en.wikipedia.org/wiki/Motion_Picture_Association_film_rating_system#Replacement_of_the_Hays_Code)</sup>.


In [105]:
dados_novos.loc[dados_novos['Classificação indicativa'] == 'Passed'].head()

,Id_Title,Título,Receita,Ano de lançamento,Classificação indicativa,Duração do filme,Gênero,Nota IMDd,Resumo,Nota Metacrict,Diretor,Estrela 1,Estrela 2,Estrela 3,Estrela 4,Número de Votos,Gênero 1,Gênero 2,Gênero 3
82,83,Sunset Blvd.,"299,645",1950,Passed,110 min,"Drama, Film-Noir",8.4,A screenwriter develops a dangerous relationsh...,NaN,Billy Wilder,William Holden,Gloria Swanson,Erich von Stroheim,Nancy Olson,201632,Drama,Noir,None
83,84,The Great Dictator,"288,475",1940,Passed,125 min,"Comedy, Drama, War",8.4,Dictator Adenoid Hynkel tries to expand his em...,NaN,Charles Chaplin,Charles Chaplin,Paulette Goddard,Jack Oakie,Reginald Gardiner,203150,Comédia,Drama,Guerra
123,124,Double Indemnity,"5,720,000",1944,Passed,107 min,"Crime, Drama, Film-Noir",8.3,An insurance representative lets himself be ta...,95.0,Billy Wilder,Fred MacMurray,Barbara Stanwyck,Edward G. Robinson,Byron Barr,143525,Crime,Drama,Noir
125,126,M - Eine Stadt sucht einen Mörder,"28,877",1931,Passed,117 min,"Crime, Mystery, Thriller",8.3,When the police in a German city are unable to...,NaN,Fritz Lang,Peter Lorre,Ellen Widmann,Inge Landgut,Otto Wernicke,143434,Crime,Mistério,Suspense
127,128,The Kid,"5,450,000",1921,Passed,68 min,"Comedy, Drama, Family",8.3,"The Tramp cares for an abandoned child, but ev...",NaN,Charles Chaplin,Charles Chaplin,Edna Purviance,Jackie Coogan,Carl Miller,113314,Comédia,Drama,Família


Apesar dos dois termos serem semelhantes, não há a necessidade de alterar agora os valores. Esses serão trocados apenas quando os filmes receberem a classificação etária brasileira.

#### 4 - Trabalhando os valores de receita

##### Receitas Faltantes

Antes de fazer a conversão dos valores de receita, existem 11 filmes que não possuem informação de valores de receita na base de dados.

In [106]:
dados[dados['Gross'].isnull()]

,Id_Title,Series_Title,Gross,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,Noofvotes
18,19,Hamilton,NaN,2020,PG-13,160 min,"Biography, Drama, History",8.6,The real life of one of America's foremost fou...,90.0,Thomas Kail,Lin-Manuel Miranda,Phillipa Soo,Leslie Odom Jr.,Renée Elise Goldsberry,55291
160,161,Hera Pheri,NaN,2000,U,156 min,"Action, Comedy, Crime",8.2,Three unemployed men look for answers to all t...,NaN,Priyadarshan,Akshay Kumar,Sunil Shetty,Paresh Rawal,Tabu,57057
166,167,Andaz Apna Apna,NaN,1994,U,160 min,"Action, Comedy, Romance",8.2,Two slackers competing for the affections of a...,NaN,Rajkumar Santoshi,Aamir Khan,Salman Khan,Raveena Tandon,Karisma Kapoor,49300
177,178,Sholay,NaN,1975,U,204 min,"Action, Adventure, Comedy",8.2,After his family is murdered by a notorious an...,NaN,Ramesh Sippy,Sanjeev Kumar,Dharmendra,Amitabh Bachchan,Amjad Khan,51284
198,199,Contratiempo,NaN,2016,TV-MA,106 min,"Crime, Drama, Mystery",8.1,A successful entrepreneur accused of murder an...,NaN,Oriol Paulo,Mario Casas,Ana Wagener,Jose Coronado,Bárbara Lennie,141516
246,247,Shin seiki Evangelion Gekijô-ban: Air/Magokoro...,NaN,1997,UA,87 min,"Animation, Action, Drama",8.1,Concurrent theatrical ending of the TV series ...,NaN,Hideaki Anno,Kazuya Tsurumaki,Megumi Ogata,Megumi Hayashibara,Yûko Miyamura,38847
323,324,Togo,NaN,2019,U,113 min,"Adventure, Biography, Drama",8.0,"The story of Togo, the sled dog who led the 19...",69.0,Ericson Core,Willem Dafoe,Julianne Nicholson,Christopher Heyerdahl,Richard Dormer,37556
451,452,Out of the Past,NaN,1947,not rated,97 min,"Crime, Drama, Film-Noir",8.0,A private eye escapes his past to run a gas st...,NaN,Jacques Tourneur,Robert Mitchum,Jane Greer,Kirk Douglas,Rhonda Fleming,32784
557,558,Harvey,NaN,1950,Approved,104 min,"Comedy, Drama, Fantasy",7.9,Due to his insistence that he has an invisible...,NaN,Henry Koster,James Stewart,Wallace Ford,William H. Lynn,Victoria Horne,52573
696,697,Badlands,NaN,1973,PG,94 min,"Action, Crime, Drama",7.8,An impressionable teenage girl from a dead-end...,93.0,Terrence Malick,Martin Sheen,Sissy Spacek,Warren Oates,Ramon Bieri,66009


Como são 11 filmes, pesquisarei um pouco sobre cada um, para entender a falta da informação no site do IMDb.

Desses filmes, 2 foram disponibilizados via serviços de streaming, não tendo assim a venda de ingressos: Hamilton e Togo.

Outros 2, Out of The Past e Harvey, foram lançados em uma época que não era comum disponibilizar esses dados. Atualmente é possível encontrar valores estimados de bilheteria para Out of The Past e de aluguéis de filmes para Harvey. 

Os outros 7 filmes sem bilheteria são filmes que nao foram produzidos nos Estados Unidos e que não tem um valor de bilheteria americana no site Box Office Mojo, afiliado ao IMDb. Alguns possuem informações de bilheteria internacional, no entanto.

Como a Alura Films deseja saber o valor de bilheteria dos gêneros, irei adicionar os valores de bilheteria internacional para os filmes possíveis. Para os filmes sem informações, deixarei os valores em branco. Essas informações serão adicionadas diretamente na ferramenta de criação de dashboards.

A conversão dos valores de receita para real também será feita durante a criação do dashboard. Os dois valores serão mostrados para a empresa, e assim ela poderá fazer estimativas independente da moeda desejada.

##### Vendo as receitas discrepantes

Além de alguns filmes não terem valores de receita, outros filmes parecem ter valores não compatíveis com a produção. Em contato com a empresa, a mesma confirmou que os valores estão corretos.

In [107]:
dados_novos['Receita'].dtype

dtype('O')

Analisando a coluna, a mesma está em formato 'string', já que os números apresentam separadores de milhar. No momento de criação do dashboard, os números serão conferidos novamente, apesar da confirmação da empresa.

#### 5 - Entendendo os valores faltantes do Metacritic.

Os valores de receita eram mais simples por existirem apenas 11 filmes dentre os 1000 sem informações. Já os valores do Metacritic são um pouco mais complicados por existirem mais de 100 filmes sem essa informação.

In [108]:
dados_novos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Id_Title                  1000 non-null   int64  
 1   Título                    1000 non-null   object 
 2   Receita                   989 non-null    object 
 3   Ano de lançamento         1000 non-null   int64  
 4   Classificação indicativa  1000 non-null   object 
 5   Duração do filme          1000 non-null   object 
 6   Gênero                    1000 non-null   object 
 7   Nota IMDd                 1000 non-null   float64
 8   Resumo                    1000 non-null   object 
 9   Nota Metacrict            843 non-null    float64
 10  Diretor                   1000 non-null   object 
 11  Estrela 1                 1000 non-null   object 
 12  Estrela 2                 1000 non-null   object 
 13  Estrela 3                 1000 non-null   object 
 14  Estrela 4

Primeiramente desejo entender o motivo da falta de avaliação do Metacritic. Pode ser uma falha no sistema do IMDb ou até um filme que não foi avaliado pelo Metacritic pois não foi lançado recentemente.

In [109]:
dados_novos[dados_novos['Nota Metacrict'].isnull()].head()

,Id_Title,Título,Receita,Ano de lançamento,Classificação indicativa,Duração do filme,Gênero,Nota IMDd,Resumo,Nota Metacrict,Diretor,Estrela 1,Estrela 2,Estrela 3,Estrela 4,Número de Votos,Gênero 1,Gênero 2,Gênero 3
20,21,Soorarai Pottru,"2,024,032",2020,U,153 min,Drama,8.6,"Nedumaaran Rajangam ""Maara"" sets out to make t...",NaN,Sudha Kongara,Suriya,Madhavan,Paresh Rawal,Aparna Balamurali,54995,Drama,None,None
54,55,Ayla: The Daughter of War,"946,947",2017,PG,125 min,"Biography, Drama, History",8.4,"In 1950, amid-st the ravages of the Korean War...",NaN,Can Ulkay,Erdem Can,Çetin Tekindor,Ismail Hacioglu,Kyung-jin Lee,34112,Biografia,Drama,História
55,56,Vikram Vedha,"546,792",2017,UA,147 min,"Action, Crime, Drama",8.4,"Vikram, a no-nonsense police officer, accompan...",NaN,Gayatri,Pushkar,Madhavan,Vijay Sethupathi,Shraddha Srinath,28401,Ação,Crime,Drama
57,58,Dangal,"12,391,761",2016,U,161 min,"Action, Biography, Drama",8.4,Former wrestler Mahavir Singh Phogat and his t...,NaN,Nitesh Tiwari,Aamir Khan,Sakshi Tanwar,Fatima Sana Shaikh,Sanya Malhotra,156479,Ação,Biografia,Drama
65,66,Taare Zameen Par,"1,223,869",2007,U,165 min,"Drama, Family",8.4,An eight-year-old boy is thought to be a lazy ...,NaN,Aamir Khan,Amole Gupte,Darsheel Safary,Aamir Khan,Tisca Chopra,168895,Drama,Família,None


Vendo os primeiros 5 filmes, constatei que eles não possuem notas no site do Metacritic. Apenas o filme Dangal possui reviews feitos por críticos no site, porém sem o número mínimo para que ele tenha uma nota. Para manter a base de dados com os filmes, irei deixar os filmes sem notas, pois incluir uma nota média geral ou por gênero pode atrapalhar análises da empresa. Caso seja necessário, depois será possível incluir alguma nota nos filmes, seja a média geral ou a média de notas dentro do seu gênero específico.

### Exportando

Após a análise dos dados e das transformações iniciais, exportarei os dados tratados para um novo arquivo do tipo .csv para criação de dashboards utilizando o Microsoft Power BI.

In [110]:
dados_novos.to_csv('dados_tratados.csv')

## Análise dos dados dos posters

### Importando

In [111]:
import pandas as pd

url = 'https://raw.githubusercontent.com/vinicius-pf/BI_Challenge_2/main/Semana%201/dataset/Posters.csv'
dados_poster = pd.read_csv(url)
dados_poster.head()

,Id_Title,Poster_Link
0,1,https://m.media-amazon.com/images/M/MV5BMDFkYT...
1,2,https://m.media-amazon.com/images/M/MV5BM2MyNj...
2,3,https://m.media-amazon.com/images/M/MV5BMTMxNT...
3,4,https://m.media-amazon.com/images/M/MV5BMWMwMG...
4,5,https://m.media-amazon.com/images/M/MV5BMWU4N2...


Para esse dataset, o mais importante é conferir se todos os filmes tem um poster.

In [113]:
dados_poster.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Id_Title     1000 non-null   int64 
 1   Poster_Link  1000 non-null   object
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


Sabemos que todos os filmes tem um poster relacionados e, em um primeiro momento, não haverá necessidade de tratamento adicional dos dados.

## Referências

1 - https://help.imdb.com/article/contribution/titles/certificates/GU757M8ZJ9ZPXB39?ref_=helpart_nav_27#

2 - https://www.quora.com/What-does-the-Metascore-score-on-IMDb-mean

3 - https://en.wikipedia.org/wiki/Motion_Picture_Association_film_rating_system#Replacement_of_the_Hays_Code

4 - https://www.gov.br/mj/pt-br/assuntos/seus-direitos/classificacao-1